In [24]:
import pandas as pd
from IPython.display import display

# CSV 파일 불러오기
value_file_path = r"C:\datasets\kor_value.csv"
ticker_file_path = r"C:\datasets\kor_ticker.csv"

data = pd.read_csv(value_file_path)
ticker_data = pd.read_csv(ticker_file_path)

# 종목코드를 문자열로 변환하여 데이터 타입을 맞춤
data['종목코드'] = data['종목코드'].astype(str)
ticker_data['종목코드'] = ticker_data['종목코드'].astype(str)

# value의 종목코드를 ticker와 같은 6자리 형식으로 변경 (.unique로 확인)
data['종목코드'] = data['종목코드'].str.zfill(6)

# 지표별 조건 필터링
per_filtered = data[(data['지표'] == 'PER') & (data['값'] <= 5) & (data['값'] > 0)]
pbr_filtered = data[(data['지표'] == 'PBR') & (data['값'] <= 0.5) & (data['값'] > 0)]
pcr_filtered = data[(data['지표'] == 'PCR') & (data['값'] <= 5) & (data['값'] > 0)]

# 조건을 모두 만족하는 종목 코드 찾기
growth_stock_codes = set(pbr_filtered['종목코드']) & set(per_filtered['종목코드']) & set(pcr_filtered['종목코드'])

# 조건을 만족하는 종목만 선택하고, 필요한 지표만 포함
filtered_data = data[(data['종목코드'].isin(growth_stock_codes)) & (data['지표'].isin(['PBR', 'PER', 'PCR']))]

# 테이블 재구성: 종목코드, 기준일을 인덱스로 두고 지표를 열로 변환
pivoted_data = filtered_data.pivot_table(index=['종목코드', '기준일'], columns='지표', values='값').reset_index()

# 종목코드 기준으로 종목명을 병합
pivoted_data = pivoted_data.merge(ticker_data[['종목코드', '종목명']], on='종목코드', how='left')

# 원하는 열 순서로 재정렬
pivoted_data = pivoted_data[['종목코드', '종목명', '기준일', 'PBR', 'PER', 'PCR']]

# 종목코드를 숫자로 변환한 후 오름차순 정렬
pivoted_data['종목코드'] = pivoted_data['종목코드'].astype(int)
pivoted_data = pivoted_data.sort_values(by='종목코드').reset_index(drop=True)

# 결과 테이블로 출력
display(pivoted_data)

# CSV 파일로 저장
output_file_path = r"C:\datasets\filtered_value_stocks.csv"
pivoted_data.to_csv(output_file_path, index=False, encoding='utf-8-sig')

# JSON 파일로 저장하기
json_file_path = r"C:\datasets\filtered_value_stocks.json"
data.to_json(json_file_path, orient="records", force_ascii=False, indent=4)

print(f"JSON 파일이 '{json_file_path}' 이름으로 저장되었습니다.")

,종목코드,종목명,기준일,PBR,PER,PCR
0,70,삼양홀딩스,2024-07-03,0.2065,2.3314,2.6540
1,140,하이트진로홀딩스,2024-07-03,0.1890,3.0299,1.7353
2,320,노루홀딩스,2024-07-03,0.2434,2.4626,1.9775
3,370,한화손해보험,2024-07-03,0.1833,2.6092,0.3606
4,540,흥국화재,2024-07-03,0.2044,1.0211,0.4023
...,...,...,...,...,...,...
81,46310,백금T&A,2024-07-03,0.4462,2.9057,1.4625
82,53700,삼보모터스,2024-07-03,0.3394,2.1764,0.7543
83,54800,아이디스홀딩스,2024-07-03,0.1154,1.9273,0.7494
84,57050,현대홈쇼핑,2024-07-03,0.1979,3.5566,2.4770


JSON 파일이 'C:\datasets\filtered_value_stocks.json' 이름으로 저장되었습니다.
